In [0]:
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("table", "")
dbutils.widgets.text("last_cdc", "")
dbutils.widgets.text("cdc_column", "")
dbutils.widgets.text("staging_path", "")

In [0]:
catalog=dbutils.widgets.get("catalog")
schema=dbutils.widgets.get("schema")
table=dbutils.widgets.get("table")
last_cdc=dbutils.widgets.get("last_cdc")
cdc_column=dbutils.widgets.get("cdc_column")
staging_path=dbutils.widgets.get("staging_path")

In [0]:
print(catalog)
print(schema)
print(table)
print(last_cdc)
print(cdc_column)
print(staging_path)

In [0]:
import json
from datetime import datetime

In [0]:
df=spark.sql(f"SELECT * FROM {catalog}.{schema}.{table} where {cdc_column} > '{last_cdc}'")

In [0]:
count=df.count()

In [0]:
if count <= 0:
    dbutils.notebook.exit(json.dumps({
    "status": "No Records",
    "records": 0,
}))

In [0]:
# df = spark.table("silver.clean.stocks")

In [0]:
df=spark.sql(f"SELECT max({cdc_column}) as max_updated_at FROM {catalog}.{schema}.{table} where {cdc_column} > '{last_cdc}'")
temp = df.collect()[0]['max_updated_at']

In [0]:
temp=temp.strftime("%Y/%m/%d %H:%M:%S")

In [0]:
# df.write.mode("overwrite").format("parquet").save("abfss://staging@singaporecomadls.dfs.core.windows.net/stage")

In [0]:
df=spark.sql(f"SELECT * FROM {catalog}.{schema}.{table} where {cdc_column} > '{last_cdc}'")

In [0]:
df.write.mode("overwrite").format("parquet").save(staging_path)

In [0]:
dbutils.notebook.exit(json.dumps({
    "status": "success",
    "records": count,
    "max_cdc":temp
}))

In [0]:
df = spark.read.parquet("abfss://staging@singaporecomadls.dfs.core.windows.net/stage")
display(df)